In [1]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from scipy.spatial.distance import cosine
from itertools import combinations
import random

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

C:\Users\a4293604\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

male_names = ["John", "Paul", "Mike", "Kevin", "Steve", "Greg", "Jeff", "Bill",
              "Frank", "George", "Ron", "Chris", "Mark", "Dan", "Tom", "Josh"]
female_names = ["Amy", "Joan", "Lisa", "Sarah", "Diana", "Kate", "Ann", "Donna",
                "Emily", "Julia", "Rachel", "Karen", "Tina", "Laura", "Jessica", "Natalie"]

career_words = ["executive", "management", "professional", "corporation", "salary", "office",
                "business", "career", "entrepreneur", "employee", "supervisor", "director",
                "consultant", "analyst", "finance", "marketing"]
family_words = ["home", "parents", "children", "family", "cousins", "marriage", "wedding",
                "relatives", "mom", "dad", "babysitter", "nursery", "housewife", "grandparents", "aunt", "uncle"]

In [11]:

def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[0][0].numpy()  # CLS token

# Apply sentence template
def apply_template(word):
    return f"This is about {word}."

def get_group_embeddings(words):
    return np.array([get_sentence_embedding(apply_template(w)) for w in words])



In [12]:
# Get embeddings
X = get_group_embeddings(male_names)
Y = get_group_embeddings(female_names)
A = get_group_embeddings(career_words)
B = get_group_embeddings(family_words)

# Differential association function
def s(w, A, B):
    return np.mean([1 - cosine(w, a) for a in A]) - np.mean([1 - cosine(w, b) for b in B])

def test_statistic(X, Y, A, B):
    return np.sum([s(x, A, B) for x in X]) - np.sum([s(y, A, B) for y in Y])

In [6]:
# Observed test statistic
observed_stat = test_statistic(X, Y, A, B)

# Permutation test
def permutation_test(X, Y, A, B, n_samples=10000, seed=42):
    random.seed(seed)
    np.random.seed(seed)
    all_embeddings = np.concatenate([X, Y])
    n = len(X)
    greater_count = 0

    for _ in range(n_samples):
        np.random.shuffle(all_embeddings)
        X_perm = all_embeddings[:n]
        Y_perm = all_embeddings[n:]
        stat = test_statistic(X_perm, Y_perm, A, B)
        if abs(stat) >= abs(observed_stat):
            greater_count += 1

    p_value = greater_count / n_samples
    return p_value

# Run test
p_value = permutation_test(X, Y, A, B, n_samples=100)
print(f"SEAT-6 Gender Bias Score: {observed_stat:.4f}")
print(f"p-value (permutation test): {p_value:.4f}")

SEAT-6 Gender Bias Score: 0.0193
p-value (permutation test): 0.0300


In [10]:
import numpy as np
from scipy.spatial.distance import cosine

# assuming you already have embeddings:
# X: embeddings for male names
# Y: embeddings for female names
# A: embeddings for career words
# B: embeddings for family words

# 1) re-define s(w,A,B)
def s(w, A, B):
    cos_with_A = [1 - cosine(w, a) for a in A]
    cos_with_B = [1 - cosine(w, b) for b in B]
    return np.mean(cos_with_A) - np.mean(cos_with_B)

# 2) collect s-values for all target words
s_X = np.array([s(x, A, B) for x in X])
s_Y = np.array([s(y, A, B) for y in Y])
all_s = np.concatenate([s_X, s_Y])

# 3) compute effect size d
mean_diff = np.mean(s_X) - np.mean(s_Y)
pooled_std = np.std(all_s, ddof=1)   # sample std-dev
cohen_d = mean_diff / pooled_std

print(f"Effect size (Cohen's d): {cohen_d:.4f}")


Effect size (Cohen's d): 0.8630
